<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/Test_bench/Test_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers


In [2]:
!pip install Keras-applications


     |████████████████████████████████| 50 kB 2.6 MB/s 


In [3]:
def select_dataset(index):

  if(index==1):
    fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
    (X_train, y_train), (X_test, y_test) = fashion_mnist

  elif(index==2):
    cifar_10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar_10
  
  elif(index==3):
    cifar_100 = tf.keras.datasets.cifar100.load_data()
    (X_train, y_train), (X_test, y_test) = cifar_100
    print(len(X_train))

    
  return X_train,y_train,X_test,y_test

In [4]:
def pre_processing_fashion_mnist(X_train,y_train,X_test,y_test,image_channel):
  img_width = X_train.shape[1]
  img_height = X_train.shape[2]
  input_shape = (img_width, img_height, image_channel)
  
  # normalize data
  X_train, X_test = X_train / 255, X_test / 255

  # reshape input 
  X_train = X_train.reshape(X_train.shape[0], *input_shape)
  X_test = X_test.reshape(X_test.shape[0], *input_shape)

  # one-hot
  y_train = tf.keras.utils.to_categorical(y_train)
  y_test = tf.keras.utils.to_categorical(y_test)

  return X_train,y_train,X_test,y_test,input_shape

def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  print(input_shape)
  return X_train,y_train,X_test,y_test,input_shape
  
def pre_processing_cifar100(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  print(input_shape)
  return X_train,y_train,X_test,y_test,input_shape

In [5]:
def define_model_vgg16(image_shape,total_classes):

  model = Sequential()
  weight_decay = 0.0005
  learning_rate = 0.1
  lr_decay = 1e-6
  lr_drop = 20
  model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=image_shape,kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))

  model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Dropout(0.5))
  model.add(Dense(total_classes))
  model.add(Activation('softmax'))
  sgd = keras.optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
  model.compile(optimizer=sgd, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
  
  return model

def define_model_standard_cnn(image_shape,total_classes):
  initializer = tf.keras.initializers.HeUniform()
  model = Sequential()
  model.add(Conv2D(32,(3,3), activation = 'relu',  padding = 'same', input_shape = image_shape))
  model.add(Conv2D(32,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))  
  model.add(Conv2D(64,(3,3), activation = 'relu',  padding = 'same'))
  model.add(Conv2D(64,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(128,(3,3), activation = 'relu',  padding = 'same'))
  model.add(Conv2D(128,(3,3), activation = 'relu',  padding = 'same'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation = 'relu'))
  model.add(Dense(total_classes, activation = 'softmax'))
  opt = keras.optimizers.SGD(learning_rate=0.001 , momentum=0.9)
  model.compile(optimizer=opt,loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
  return model



In [12]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def ResNet34(shape, classes):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
'''def identity_block(layer,f,nfilters):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1),padding='valid')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)

def Conv_block(layer,f,nfilters,s=2):
    Xshortcut=layer
    F1,F2,F3=nfilters
    layer=Conv2D(F1,(1,1),strides=(s,s))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F2,(f,f),padding='same')(layer)
    layer=BatchNormalization(axis=-1)(layer)
    layer=Activation('relu')(layer)
    
    layer=Conv2D(F3,(1,1))(layer)
    layer=BatchNormalization(axis=-1)(layer)
    
    Xshortcut=Conv2D(F3,(1,1),strides=(s,s))(Xshortcut)
    Xshortcut=BatchNormalization(axis=-1)(Xshortcut)
    
    layer=Add()([Xshortcut,layer])
    layer=Activation('relu')(layer)
    return(layer)

def Resnet50(img_shape,numclasses):
    img_inp=Input(shape=img_shape,dtype=tf.float32)
    X=ZeroPadding2D((3,3))(img_inp)
    X=Conv2D(64,(7,7),strides=(2,2))(X)
    X=BatchNormalization(axis=-1)(X)
    X=Activation('relu')(X)
    
    X=MaxPooling2D((3,3),strides=(2,2),padding='same')(X)
    X=Conv_block(X,3,[64,64,256],s=1)
    X=identity_block(X,3,[64,64,256])
    X=identity_block(X,3,[64,64,256])
    
    X=Conv_block(X,3,[128,128,512],s=2)
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    X=identity_block(X,3,[128,128,512])
    
    X=Conv_block(X,3,[256,256,1024],s=2)
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    X=identity_block(X,3,[256,256,1024])
    
    X=Conv_block(X,3,[512,512,2048],s=2)
    X=identity_block(X,3,[512,512,2048])
    X=identity_block(X,3,[512,512,2048])
    
    X=AveragePooling2D((7,7) , padding='same')(X)
    
    X=Flatten()(X)
    X=Dense(numclasses,activation='softmax')(X)
    model=Model(inputs=img_inp,outputs=X)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return(model)'''

"def identity_block(layer,f,nfilters):\n    Xshortcut=layer\n    F1,F2,F3=nfilters\n    layer=Conv2D(F1,(1,1),padding='valid')(layer)\n    layer=BatchNormalization(axis=-1)(layer)\n    layer=Activation('relu')(layer)\n    \n    layer=Conv2D(F2,(f,f),padding='same')(layer)\n    layer=BatchNormalization(axis=-1)(layer)\n    layer=Activation('relu')(layer)\n    \n    layer=Conv2D(F3,(1,1),padding='valid')(layer)\n    layer=BatchNormalization(axis=-1)(layer)\n    \n    layer=Add()([Xshortcut,layer])\n    layer=Activation('relu')(layer)\n    return(layer)\n\ndef Conv_block(layer,f,nfilters,s=2):\n    Xshortcut=layer\n    F1,F2,F3=nfilters\n    layer=Conv2D(F1,(1,1),strides=(s,s))(layer)\n    layer=BatchNormalization(axis=-1)(layer)\n    layer=Activation('relu')(layer)\n    \n    layer=Conv2D(F2,(f,f),padding='same')(layer)\n    layer=BatchNormalization(axis=-1)(layer)\n    layer=Activation('relu')(layer)\n    \n    layer=Conv2D(F3,(1,1))(layer)\n    layer=BatchNormalization(axis=-1)(layer)\

In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
def train_model_vgg16(X_train,y_train,X_test,y_test,epochs=10,batch_size=128):
  
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_vgg16', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size,callbacks = callbacks, verbose = 1,validation_data=(X_test,y_test))
  return history

def train_model_standard_cnn(X_train,y_train,X_test,y_test,epochs=10,batch_size=64):
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_standard_cnn', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train , y_train , epochs = epochs , validation_data=(X_test,y_test),batch_size=batch_size , callbacks = callbacks)
  return history

def train_model_resnet50(X_train,y_train,X_test,y_test,epochs=10):
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_resnet50', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train , y_train , epochs = epochs , validation_data=(X_test,y_test),batch_size= 64 , callbacks = callbacks)
  return history

def train_model_resnet34(X_train,y_train,X_test,y_test,epochs=10):
  callbacks = [ 
    tf.keras.callbacks.ModelCheckpoint('best_model_resnet50', monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.1, patience = 3, min_lr = 0.00001, verbose = 1 )
  ]
  history=model.fit(X_train , y_train , epochs = epochs , validation_data=(X_test,y_test),batch_size= 64 , callbacks = callbacks)
  return history

In [10]:
def make_prediction(model,X_test,index):

  return model.predict(X_test(index))


In [ ]:

'''X_train,y_train,X_test,y_test=select_dataset(1)
X_train,y_train,X_test,y_test,image_shape=pre_processing_fashion_mnist(X_train,y_train,X_test,y_test,1)
model=define_model(image_shape,10)
train_model(X_train,y_train,X_test,y_test)'''

'''X_train,y_train,X_test,y_test=select_dataset(2)
X_train,y_train,X_test,y_test,image_shape=pre_processing_cifar10(X_train,y_train,X_test,y_test,3)
model=define_model_standard_cnn(image_shape,10)
train_model_standard_cnn(X_train,y_train,X_test,y_test,epochs=75)'''

'''X_train,y_train,X_test,y_test=select_dataset(2)
X_train,y_train,X_test,y_test,image_shape=pre_processing_cifar100(X_train,y_train,X_test,y_test,3)
model=Resnet50(image_shape,10)
history=train_model_resnet50(X_train,y_train,X_test,y_test,epochs=100)'''

X_train,y_train,X_test,y_test=select_dataset(2)
X_train,y_train,X_test,y_test,image_shape=pre_processing_cifar10(X_train,y_train,X_test,y_test,3)
model=ResNet34(image_shape,10)
history=train_model_resnet34(X_train,y_train,X_test,y_test,epochs=100)

(32, 32, 3)
Epoch 1/100
782/782 [==============================] - 129s 122ms/step - loss: 1.5428 - accuracy: 0.4353 - val_loss: 1.8306 - val_accuracy: 0.4100

Epoch 00001: val_accuracy improved from -inf to 0.41000, saving model to best_model_resnet50
INFO:tensorflow:Assets written to: best_model_resnet50/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
782/782 [==============================] - 95s 121ms/step - loss: 1.1155 - accuracy: 0.6006 - val_loss: 1.1330 - val_accuracy: 0.5997

Epoch 00002: val_accuracy improved from 0.41000 to 0.59970, saving model to best_model_resnet50
INFO:tensorflow:Assets written to: best_model_resnet50/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 3/100
782/782 [==============================] - 95s 121ms/step - loss: 0.9218 - accuracy: 0.6761 - val_loss: 1.0473 - val_accuracy: 0.6420

Epoch 00003: val_accuracy improved from 0.59970 to 0.64200, saving model to best_model_resnet50
INFO:tensorflow:Assets written to: best_model_resnet50/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 4/100
782/782 [==============================] - 95s 121ms/step - loss: 0.7990 - accuracy: 0.7205 - val_loss: 1.0539 - val_accuracy: 0.6623

Epoch 00004: val_accuracy improved from 0.64200 to 0.66230, saving model to best_model_resnet50
INFO:tensorflow:Assets written to: best_model_resnet50/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 5/100
406/782 [==============>...............] - ETA: 43s - loss: 0.6777 - accuracy: 0.7656

In [ ]:
import matplotlib.pyplot as plt

score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
!pip freeze > requirements.txt